# Anomaly Detection and Impact Extraction

This notebook demonstrates the process of detecting anomalies in microservice systems and extracting their impact topologies. The workflow includes:
1. **Data Loading**: Load monitoring data from calling relationships
2. **Data Preprocessing**: Organize service pair data and add historical context
3. **Anomaly Detection**: Identify anomalous behavior in service communications
4. **Impact Extraction**: Extract topology structures representing incident impacts

The output will be raw topology data that can be used for further analysis and incident diagnosis.

## Step 1: Import Required Libraries

We import essential libraries for data processing, clustering, visualization, and graph analysis.

In [1]:
# Data processing and analysis libraries
import pandas as pd
import numpy as np

# Machine learning for clustering anomalous patterns
from sklearn.cluster import DBSCAN

# Visualization and graph analysis
import matplotlib.pyplot as plt
import networkx as nx

# Data serialization and file I/O
import json
import pickle

## Step 2: Load Monitoring Data

Load the calling relationships monitoring data which contains:
- **TimeStamp**: When the measurement was taken
- **SourceName**: The calling service
- **DestinationName**: The called service
- **Workload**: Number of requests/calls
- **FailCount**: Number of failed requests

In [2]:
# Load the monitoring data containing service call relationships and metrics
all_data = pd.read_csv("../data/calling_relationships_monitoring.csv")

## Step 3: Explore the Dataset

Display the dataset to understand its structure and content. This helps us verify the data format and identify any potential issues.

In [3]:
# Display the dataset overview
# This shows us the structure: timestamps, service pairs, workloads, and failure counts
all_data

,TimeStamp,SourceName,DestinationName,Workload,FailCount
0,2022-03-23 15:55:00,frontend,adservice,664.000000,0.000000
1,2022-03-23 15:55:00,frontend,checkoutservice,54.666667,0.000000
2,2022-03-23 15:55:00,frontend,shippingservice,250.666667,0.000000
3,2022-03-23 15:55:00,frontend,currencyservice,3653.333333,0.000000
4,2022-03-23 15:55:00,frontend,productcatalogservice,5172.000000,0.000000
...,...,...,...,...,...
281355,2022-04-04 20:59:00,checkoutservice,productcatalogservice,104.000000,0.000000
281356,2022-04-04 20:59:00,checkoutservice,cartservice,98.666667,0.000000
281357,2022-04-04 20:59:00,recommendationservice,productcatalogservice,1001.333333,20.000000
281358,2022-04-04 20:59:00,cartservice,redis-cart,736.000000,0.000000


## Step 4: Organize Data by Service Pairs

Group the data by service pairs (source-destination combinations) and add historical context:
- Create separate time series for each service pair
- Add yesterday's failure count as a baseline for comparison (shift by 1440 minutes = 24 hours)
- This historical context helps identify unusual patterns

In [4]:
# Create a dictionary to store time series data for each service pair
ServicePairs = {}

# Group data by source and destination service names
for g in all_data.groupby(['SourceName', 'DestinationName']):
    # g[0] contains the group key (source, destination tuple)
    # g[1] contains the grouped data for this service pair
    ServicePairs[g[0]] = g[1].reset_index(drop=True)
    
    # Add historical context: yesterday's failure count for comparison
    # Shift by 1440 minutes (24 hours) to get the same time yesterday
    # Fill missing values with 0 for the first day
    ServicePairs[g[0]]['YesterFailCount'] = ServicePairs[g[0]]['FailCount'].shift(1440).fillna(0)

# Anomaly Detection

This section implements a simple anomaly detection mechanism. In production environments, more sophisticated anomaly detectors would be used for better performance.

## Step 5: Simple Anomaly Detection

**Note**: This is a simplified anomaly detector for demonstration purposes. In practice, more advanced methods would be used.

The current approach:
- Considers any non-zero failure count as an anomaly
- This is suitable for this simulation dataset where failures are explicitly injected
- Real-world systems would use statistical methods, machine learning, or threshold-based approaches

In [5]:
# Create anomaly indicators for each service pair
ServicePairsAnomalies = {}

for pair in ServicePairs:
    # Simple anomaly detection: any failure count > 0 is considered anomalous
    # This works for simulation data where failures are explicitly injected
    ServicePairsAnomalies[pair] = ServicePairs[pair]['FailCount'] > 0
    
    # Alternative approach (commented): require consecutive anomalies
    # This would reduce false positives by requiring sustained anomalous behavior
    # ServicePairsAnomalies[pair] = pd.Series(np.all([
    #     ServicePairsAnomalies[pair],
    #     ServicePairsAnomalies[pair].shift(1), 
    #     ServicePairsAnomalies[pair].shift(2)
    # ], axis=0))

## Step 6: Prepare System-Level Anomaly Data

Combine individual service pair anomalies into a system-wide view to identify time periods when the system experiences issues.

In [6]:
# Get list of all service pair keys for processing
keys = list(ServicePairs.keys())

In [7]:
# Get the length of time series data (should be same for all service pairs)
datalen = len(ServicePairs[keys[0]])

In [8]:
# Create a system-wide anomaly matrix
# Each row represents a time point, each column represents a service pair
# True indicates an anomaly in that service pair at that time
SystemAnomalies = pd.concat([ServicePairsAnomalies[k] for k in ServicePairsAnomalies], axis=1)

In [9]:
# Set column names to service pair identifiers
SystemAnomalies.columns = [k for k in ServicePairsAnomalies]

## Step 7: Analyze System-Level Anomaly Distribution

Check how many time periods have at least one anomalous service pair. This gives us an overview of system health over time.

In [10]:
# Count time periods with and without system-level anomalies
# any(axis=1) returns True if any service pair has an anomaly at that time
SystemAnomalies.any(axis=1).value_counts()

True     11592
False     5993
Name: count, dtype: int64

In [11]:
# Create a boolean series indicating time periods with any system anomaly
# This will be used to identify when to extract impact topologies
SystemAnomalies_any = SystemAnomalies.any(axis=1)

# Impact Extraction

This section extracts topology structures that represent the impact of incidents on the system. The process identifies connected components of anomalous service pairs and creates topology features for analysis.

## Step 8: Extract Impact Topologies

For each time period with anomalies, extract the topology structure representing the incident impact:

**Process Overview:**
1. **Single Edge Case**: If only one service pair is anomalous, create a simple topology
2. **Multiple Edges Case**: Use correlation analysis and clustering to group related anomalies
3. **Graph Analysis**: Use NetworkX to find connected components in the anomaly graph
4. **Feature Extraction**: Extract time series features for each topology

In [12]:
# Configuration parameters for topology extraction
THRESHOLD = 0.9  # Correlation threshold for grouping related anomalies
min_sample = 1   # Minimum samples for DBSCAN clustering

# List to store all extracted topologies
Topologies = []

# Process each time point (starting from index 9 to have enough historical data)
for t in range(9, datalen-1):
    # Only process time points where system has anomalies
    if SystemAnomalies_any.loc[t]:
        # Get list of service pairs that are anomalous at this time
        anomalypairs = [k for k in keys if SystemAnomalies.loc[t][k]]
        
        # Case 1: Single anomalous service pair
        if len(anomalypairs) == 1:
            # Create topology feature structure
            topoFea = {}
            topoFea['time'] = t
            topoFea['edges_info'] = []
            
            edge = anomalypairs[0]
            topoedge = {}
            topoedge['src'] = edge[0]  # Source service
            topoedge['des'] = edge[1]  # Destination service
            
            # Extract 10-minute time window of metrics (t-9 to t)
            topoedge['FailCount'] = ServicePairs[edge].loc[t-9:t]['FailCount'].tolist()
            topoedge['Workload'] = ServicePairs[edge].loc[t-9:t]['Workload'].tolist()
            topoedge['YesterFailCount'] = ServicePairs[edge].loc[t-9:t]['YesterFailCount'].tolist()
            
            topoFea['edges_info'].append(topoedge)
            topoFea['MaxFail'] = topoedge['FailCount'][-1]  # Current failure count
            topoFea['nodes'] = [edge[0], edge[1]]  # Services involved
            topoFea['TimeStamp'] = ServicePairs[edge].loc[t]['TimeStamp']
            
            Topologies.append(topoFea)
            
        # Case 2: Multiple anomalous service pairs
        elif len(anomalypairs) > 1:
            # Extract failure count patterns for correlation analysis
            point_list = [ServicePairs[pair].loc[t-9:t]['FailCount'].tolist() for pair in anomalypairs]
            
            # Calculate correlation matrix between anomaly patterns
            distance_matrix = np.corrcoef(point_list)
            distance_matrix[np.isnan(distance_matrix)] = 0  # Handle NaN values
            
            # Set diagonal to 1 (perfect self-correlation)
            idx = [idx for idx in range(len(distance_matrix))]
            distance_matrix[idx, idx] = 1
            
            # Convert correlation to distance for clustering
            distance_matrix = np.abs(distance_matrix)
            distance_matrix[distance_matrix >= THRESHOLD] = 1  # High correlation = close
            distance_matrix[distance_matrix < THRESHOLD] = 2   # Low correlation = far
            
            # Use DBSCAN to cluster related anomalies
            y_pred = DBSCAN(eps=1.5, min_samples=min_sample, metric='precomputed').fit_predict(distance_matrix).tolist()
            
            # Group anomalous pairs by cluster
            clusters = [[] for i in range(max(y_pred)+1)]
            for i, ano_pair in enumerate(anomalypairs):
                clusters[y_pred[i]].append(ano_pair)
            
            # Process each cluster to extract connected topologies
            for cluster in clusters:
                # Create undirected and directed graphs
                g = nx.Graph()        # For finding connected components
                di_g = nx.DiGraph()   # For preserving edge directions
                
                g.add_edges_from(cluster)
                di_g.add_edges_from(cluster)
                
                # Extract each connected component as a separate topology
                for sub_g in nx.connected_components(g):
                    topoFea = {}
                    topoFea['time'] = t
                    topoFea['edges_info'] = []
                    MaxFail = 0
                    
                    # Extract features for each edge in this topology
                    for edge in list(di_g.subgraph(sub_g).edges):
                        topoedge = {}
                        topoedge['src'] = edge[0]
                        topoedge['des'] = edge[1]
                        
                        # Extract time series features
                        topoedge['FailCount'] = ServicePairs[edge].loc[t-9:t]['FailCount'].tolist()
                        topoedge['Workload'] = ServicePairs[edge].loc[t-9:t]['Workload'].tolist()
                        topoedge['YesterFailCount'] = ServicePairs[edge].loc[t-9:t]['YesterFailCount'].tolist()
                        
                        topoFea['edges_info'].append(topoedge)
                        MaxFail = max(MaxFail, topoedge['FailCount'][-1])
                    
                    topoFea['MaxFail'] = MaxFail
                    topoFea['nodes'] = list(sub_g)
                    topoFea['TimeStamp'] = ServicePairs[edge].loc[t]['TimeStamp']
                    Topologies.append(topoFea)

## Step 9: Verify Extraction Results

Check the number of topologies extracted. This gives us an idea of how many potential incidents were identified in the dataset.

In [13]:
# Display the total number of extracted topologies
# Each topology represents a potential incident impact structure
len(Topologies)

26132

## Step 10: Save Raw Topologies

Save the extracted topologies to a pickle file for use in subsequent analysis steps. This raw data will be processed further for labeling and feature engineering.

In [14]:
# Save the extracted topologies for further processing
# These raw topologies will be used in the next steps for:
# 1. Data labeling (identifying true incidents)
# 2. Feature engineering (creating ML-ready features)
# 3. Model training and evaluation
with open('../data/raw_topoloies.pkl', 'wb') as f:
    pickle.dump(Topologies, f)